In [34]:
%run 'Common.ipynb'
fname = os.path.join ('data','2015-05-02','ArterialStiffnessRatData-Summary-Split.tsv')
colrange = range (2,1000)
#cardio_traits = np.loadtxt(fname, delimiter = "\t", skiprows = 1, usecols = colrange)
cardio_data = np.loadtxt(fname, delimiter = "\t", skiprows = 1)
contig_traits = cardio_data[:,2:]
contig_class_vals = cardio_data[:,1]
class_traits, class_vals = get_class_mat_list (contig_traits, contig_class_vals)
print (contig_traits)
print
print (class_traits)
print
print (contig_class_vals)
print
print (class_vals)
print 

[[ 88.81367     2.07774    89.00051   ...,   0.5159692   0.7739539
    1.031938 ]
 [ 77.18535     2.274467   77.34305   ...,   4.822376    7.233564
    9.644752 ]
 [ 91.70094     1.639331   92.12057   ...,   0.5158894   0.7738341
    1.031779 ]
 ..., 
 [ 90.62921     2.31012    90.55712   ...,   0.5180005   0.7770008
    1.036001 ]
 [ 83.66883     1.706524   83.55207   ...,   2.675371    4.013056
    5.350741 ]
 [ 91.05768     2.41444    90.82329   ...,   1.515305    2.272957
    3.030609 ]]

[[[ 88.81367     2.07774    89.00051   ...,   0.5159692   0.7739539
     1.031938 ]
  [ 77.18535     2.274467   77.34305   ...,   4.822376    7.233564
     9.644752 ]
  [ 91.70094     1.639331   92.12057   ...,   0.5158894   0.7738341
     1.031779 ]
  [ 91.31534     3.085426   91.79221   ...,   2.516293    3.774439
     5.032586 ]
  [ 89.53819     2.327437   89.253     ...,   0.5181079   0.7771618
     1.036216 ]]

 [[ 86.47728     2.330642   86.7679    ...,   2.647884    3.971827
     5.295769 ]

In [52]:
nfeatures = 60
niter = np.product ( [m.shape[0] for m in class_traits] )
n_correct = np.asarray( [0]*2)
(train,test) = round_robin_iteration (0,class_traits)
print ('Train class sizes : {}'.format([x.shape[0] for x in train]))
print ('Features          : {}'.format(nfeatures))
print ('Iterations        : {}'.format(niter))
for iter_idx in range ( niter ):
    # Split
    (train,test) = round_robin_iteration (iter_idx,class_traits)
    (contig_train_mat, contig_train_vals) = list_to_contig_mat (train, class_vals)
    (contig_test_mat, contig_test_vals) = list_to_contig_mat (test, class_vals)

    # Normalize
    (norm_train, norm_test) = normalize (contig_train_mat, contig_test_mat)
    
    # Reduce
    feature_weights = Pearson(norm_train, contig_train_vals)
    (sorted_train, sorted_test) = weigh_sort (norm_train, norm_test, feature_weights)

    # Classify
#    preds,pred_vals = WND5(sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals)

    preds = rand_forest_clf (sorted_train[:,:nfeatures], sorted_test[:,:nfeatures], contig_train_vals, iter_idx)
    for pred_idx in range (len(preds)):
        if (preds[pred_idx] == contig_test_vals[pred_idx]):
            n_correct[pred_idx] += 1
    cumul_acc = [(float(x) / (float(iter_idx)+1.0)) for x in n_correct]
    print ('Iteration {}; Predictions: {}; cumulative accuracies: {}'.format(iter_idx, preds, cumul_acc))


Train class sizes : [4, 4]
Features          : 60
Iterations        : 25
Iteration 0; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 1; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 2; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 3; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 4; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 5; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 6; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 7; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 8; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 9; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 10; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 11; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
Iteration 12; Predictions: [ 2.  1.]; cumulative accuracies: [1.0, 1.0]
I